# 数据处理

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("E:/desktop/2023win/visualization/DATA/Car/US_Accidents_March23.csv")

In [3]:
print("Size of our Dataset:", df.shape)

Size of our Dataset: (7728394, 46)


In [4]:
pd.set_option('display.max_columns', None)
df.head(3)

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Street,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day


In [7]:
from pprint import pprint
def sanity_check(df):
    pd.set_option('display.max_columns', None)
    pprint('-'*70)
    pprint('No. of Rows: {0[0]}        No. of Columns : {0[1]}'.format(df.shape))
    pprint('-'*70)
    data_profile = pd.DataFrame(df.dtypes.reset_index()).rename(columns = {'index':'Attribute', 0:'DataType'}).set_index('Attribute')
    
    data_profile = pd.concat([data_profile,df.isnull().sum()], axis=1).rename(columns = {0 : 'Missing Values'})
    data_profile = pd.concat([data_profile,(df.isnull().mean()*100).round(2)], axis=1).rename(columns = {0 : 'Missing %'})
    data_profile = pd.concat([data_profile,df.nunique()], axis=1).rename(columns = {0 : 'Unique Values'})
    
    pprint(data_profile)
    pprint('-'*70)

sanity_check(df)

'----------------------------------------------------------------------'
'No. of Rows: 7426729        No. of Columns : 46'
'----------------------------------------------------------------------'
                      DataType  Missing Values  Missing %  Unique Values
ID                      object               0       0.00        7426729
Source                  object               0       0.00              3
Severity                 int64               0       0.00              4
Start_Time              object               0       0.00        5926304
End_Time                object               0       0.00        6470695
Start_Lat              float64               0       0.00        2347656
Start_Lng              float64               0       0.00        2396804
End_Lat                float64         3301235      44.45        1505858
End_Lng                float64         3301235      44.45        1540815
Distance(mi)           float64               0       0.00          21834
D

In [9]:
# # Specify the names of the columns to be dropped
# cols2drop = ['End_Lat', 'End_Lng', 'ID', 'Source', 'Airport_Code', 'Weather_Timestamp', 'Timezone']

# # Use the drop() method to remove the specified columns
# df.drop(columns=cols2drop, inplace=True)

## 去除不感兴趣数据

In [6]:
# Specify the names of the columns to be dropped
cols2drop = ['End_Lat', 'End_Lng', 'ID', 'Source', 'Airport_Code', 'Weather_Timestamp', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight', 'Timezone']

# Use the drop() method to remove the specified columns
df.drop(columns=cols2drop, inplace=True)

In [3]:
# 去除国家不是美国的记录
df = df[df['Country'] == 'US']

In [10]:
sanity_check(df)

'----------------------------------------------------------------------'
'No. of Rows: 7426729        No. of Columns : 39'
'----------------------------------------------------------------------'
                      DataType  Missing Values  Missing %  Unique Values
Severity                 int64               0        0.0              4
Start_Time              object               0        0.0        5926304
End_Time                object               0        0.0        6470695
Start_Lat              float64               0        0.0        2347656
Start_Lng              float64               0        0.0        2396804
Distance(mi)           float64               0        0.0          21834
Description             object               0        0.0        3632562
Street                  object               0        0.0         327345
City                    object               0        0.0          12237
County                  object               0        0.0           1813
S

## 处理缺失值

In [4]:
# 缺失值小于5%的删除
df.dropna(subset=['Visibility(mi)', 'Wind_Direction', 'Description', 'Humidity(%)', 'Weather_Condition', 'Temperature(F)', 'Pressure(in)', 'Sunrise_Sunset', 'Street', 'Zipcode'], inplace=True)

In [5]:
# 用平均值填充缺失值大于5%的数据
columns = ['Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)']

for c in columns:
    df[c].fillna(df[c].mean(), inplace=True)

In [8]:
sanity_check(df)

'----------------------------------------------------------------------'
'No. of Rows: 7426729        No. of Columns : 46'
'----------------------------------------------------------------------'
                      DataType  Missing Values  Missing %  Unique Values
ID                      object               0       0.00        7426729
Source                  object               0       0.00              3
Severity                 int64               0       0.00              4
Start_Time              object               0       0.00        5926304
End_Time                object               0       0.00        6470695
Start_Lat              float64               0       0.00        2347656
Start_Lng              float64               0       0.00        2396804
End_Lat                float64         3301235      44.45        1505858
End_Lng                float64         3301235      44.45        1540815
Distance(mi)           float64               0       0.00          21834
D

## 去除重复值

In [11]:
# 去除重复值
print("Number of rows:", len(df.index))
df.drop_duplicates(inplace=True)
print("Number of rows after dropping duplicates:", len(df.index))

Number of rows: 7426729
Number of rows after dropping duplicates: 7329850


## 后加处理

In [14]:
import json

twilight_fields = ['Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']

# 初始化一个列表来存储结果
result_list = []

# 遍历每个字段，统计 'night' 和 'day' 的数量
for field in twilight_fields:
    night_count = df[df[field] == 'Night'].shape[0]
    day_count = df[df[field] == 'Day'].shape[0]
    
    # 构造结果字典
    result_dict = {
        "name": field,
        "value": night_count + day_count,
        "children": [
            {"name": "Night", "value": night_count},
            {"name": "Day", "value": day_count}
        ]
    }

    result_list.append(result_dict)

# 将结果列表转换为JSON格式
json_result = json.dumps(result_list, indent=4)

# 打印JSON结果
print(json_result)

[
    {
        "name": "Sunrise_Sunset",
        "value": 7329850,
        "children": [
            {
                "name": "Night",
                "value": 2225490
            },
            {
                "name": "Day",
                "value": 5104360
            }
        ]
    },
    {
        "name": "Civil_Twilight",
        "value": 7329850,
        "children": [
            {
                "name": "Night",
                "value": 1880175
            },
            {
                "name": "Day",
                "value": 5449675
            }
        ]
    },
    {
        "name": "Nautical_Twilight",
        "value": 7329850,
        "children": [
            {
                "name": "Night",
                "value": 1516708
            },
            {
                "name": "Day",
                "value": 5813142
            }
        ]
    },
    {
        "name": "Astronomical_Twilight",
        "value": 7329850,
        "children": [
            {
          

## 后加处理结束

In [13]:
# 将 DataFrame 保存到另一个 CSV 文件
df.to_csv("E:/desktop/2023win/visualization/DATA/Car/US_Accidents_clean.csv", index=False)# 将 DataFrame 保存到另一个 CSV 文件

# 数据计算与分析

In [4]:
df = pd.read_csv("E:/desktop/2023win/visualization/DATA/Car/US_Accidents_clean.csv")

## 数量统计

In [27]:
# 计算每个州的车祸总数
state_counts = df["State"].value_counts()

In [28]:
state_counts_df = pd.DataFrame({'State': state_counts.index, 'count': state_counts.values})

In [32]:
# 将州的车祸数量转换为 DataFrame
states = pd.DataFrame(state_counts).reset_index()

# 打印重命名前的列名
print("重命名前的列名:")
print(states.columns)

# 重命名列
states.rename(columns={'index': 'state_code', 'State': 'cases'}, inplace=True)

# 打印重命名后的列名
print("\n重命名后的列名:")
print(states.columns)

# 按照车祸数量排序 DataFrame
states = states.sort_values(by='cases', ascending=False)

# 检查是否有重复值
duplicates = states[states.duplicated(subset=['state_code'], keep=False)]

# 打印排序后的 DataFrame
print("\n排序后的 DataFrame:")
print(states.head())

# 打印任何重复值
print('\n重复值:')
print(duplicates)


重命名前的列名:
Index(['index', 'State'], dtype='object')

重命名后的列名:
Index(['state_code', 'cases'], dtype='object')

排序后的 DataFrame:
  state_code    cases
0         CA  1651043
1         FL   838319
2         TX   562295
3         SC   368624
4         NY   331885

重复值:
Empty DataFrame
Columns: [state_code, cases]
Index: []


In [33]:
# # convert the states series into a DataFrame
# states = pd.DataFrame(state_counts).reset_index()

# # rename the columns
# states.rename(columns={'State': 'state_code', 'count': 'cases'}, inplace=True)

# sort the DataFrame by cases (the no. of accidents)
states = states.sort_values(by='cases', ascending=False)

# print the sorted DataFrame
states.head()


,state_code,cases
0,CA,1651043
1,FL,838319
2,TX,562295
3,SC,368624
4,NY,331885


In [34]:
# 换掉简称
us_states = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts',
    'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
    'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico',
    'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Add a new column 'State_Name' based on 'State_Code'
states['state'] = states['state_code'].map(us_states)

# Display the updated DataFrame
states.head()

,state_code,cases,state
0,CA,1651043,California
1,FL,838319,Florida
2,TX,562295,Texas
3,SC,368624,South Carolina
4,NY,331885,New York


In [37]:
states.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/state_count.csv", index=False)

In [35]:
top_10 = states[:10]

In [36]:
top_10.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/state_top_10.csv", index=False)

In [4]:
# 每个城市
cities = pd.DataFrame(df["City"].value_counts()).reset_index().sort_values(by='City',ascending=False)
cities = cities.rename(columns={'index':'city','City':'cases'})

In [5]:
cities.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/cities_count.csv", index=False)

## *时间统计*

In [6]:
# 将 "Start_Time" 列转换为 datetime 类型
df["Start_Time"] = pd.to_datetime(df["Start_Time"], format="%Y-%m-%d %H:%M:%S", errors='coerce', dayfirst=True)

# 将 "End_Time" 列转换为 datetime 类型
df["End_Time"] = pd.to_datetime(df["End_Time"], format="%Y-%m-%d %H:%M:%S", errors='coerce', dayfirst=True)

# 提取年、月、星期几、日期和小时信息
df["Year"] = df["Start_Time"].dt.year
df["Month"] = df["Start_Time"].dt.month
df["Weekday"] = df["Start_Time"].dt.weekday
df["Day"] = df["Start_Time"].dt.day
df["Hour"] = df["Start_Time"].dt.hour


In [13]:
df.head(5)

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,County,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Year,Month,Weekday,Day,Hour
0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,0.01,Right lane blocked due to accident on I-70 Eas...,I-70 E,Dayton,Montgomery,...,False,False,False,False,Night,2016,2,0,8,5
1,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,Brice Rd,Reynoldsburg,Franklin,...,False,False,False,False,Night,2016,2,0,8,6
2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,0.01,Accident on OH-32 State Route 32 Westbound at ...,State Route 32,Williamsburg,Clermont,...,False,False,True,False,Night,2016,2,0,8,6
3,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,0.01,Accident on I-75 Southbound at Exits 52 52B US...,I-75 S,Dayton,Montgomery,...,False,False,False,False,Night,2016,2,0,8,7
4,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Miamisburg Centerville Rd,Dayton,Montgomery,...,False,False,True,False,Day,2016,2,0,8,7


In [18]:
year_df = pd.DataFrame(df['Year'].value_counts()).reset_index().sort_values(by='Year', ascending=True)
year = year_df.rename(columns={'index':'year','Year':'cases'})

In [20]:
year.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/year_count.csv", index=False)

### *month*

In [23]:
month_df = pd.DataFrame(df.Start_Time.dt.month.value_counts()).reset_index()
month = month_df.rename(columns={'index':'month#','month#':'cases'}).sort_values(by='month#', ascending=True)

# adding month name as a column
month_map = {1:'Jan' , 2:'Feb' , 3:'Mar' , 4:'Apr' , 5:'May' , 6:'Jun', 7:'Jul' , 8:'Aug', 9:'Sep',10:'Oct' , 11:'Nov' , 12:'Dec'}
month['month_name'] = month['month#'].map(month_map)

In [25]:
month.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/month_count.csv", index=False)

### *周*

In [26]:
dow = pd.DataFrame(df['Start_Time'].dt.dayofweek.value_counts()).reset_index()
dow = dow.rename(columns={'index':'day_of_week', 'Start_Time':'cases'}).sort_values(by='day_of_week')
day_map = {0:'Monday' , 1:'Tuesday' , 2:'Wednesday' , 3:"Thursday" , 4:'Friday' , 5:"Saturday" , 6:'Sunday'}   
dow['weekday'] = dow['day_of_week'].map(day_map)

In [27]:
dow.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/dow_count.csv", index=False)

### *天*

In [29]:
# 24时间
hour_of_day = pd.DataFrame(df['Hour'].value_counts()).reset_index().rename(columns={'index':'hour','Hour':'cases'})
hour_of_day.sort_values(by='hour', inplace=True)

In [31]:
# 24时间
hour_of_day.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/hour_of_day.csv", index=False)

### *等级*

In [19]:
# Calculate the percentage of each severity level
severity = df['Severity'].value_counts(normalize=True).round(2) * 100

In [21]:
severity.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/severity_all.csv", index=False)

In [17]:
df_subset = df[['Year', 'Severity']]

# 使用 groupby 计算每年各严重程度的事故数量
accidents_by_year_severity = df_subset.groupby(['Year', 'Severity']).size().unstack()

In [22]:
accidents_by_year_severity.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/year_severity.csv", index=False)

### *weather*

In [23]:
print("No. of Weather Conditions:", len(df["Weather_Condition"].unique()))

# To view the complete list of 142 weather descriptions, run the following code
print("\nList of unique weather conditions:", list(df["Weather_Condition"].unique()))

No. of Weather Conditions: 142

List of unique weather conditions: ['Light Rain', 'Overcast', 'Mostly Cloudy', 'Rain', 'Light Snow', 'Haze', 'Scattered Clouds', 'Partly Cloudy', 'Clear', 'Snow', 'Light Freezing Drizzle', 'Light Drizzle', 'Fog', 'Shallow Fog', 'Heavy Rain', 'Light Freezing Rain', 'Cloudy', 'Drizzle', 'Light Rain Showers', 'Mist', 'Smoke', 'Patches of Fog', 'Light Freezing Fog', 'Light Haze', 'Light Thunderstorms and Rain', 'Thunderstorms and Rain', 'Fair', 'Volcanic Ash', 'Blowing Sand', 'Blowing Dust / Windy', 'Widespread Dust', 'Fair / Windy', 'Rain Showers', 'Light Rain / Windy', 'Hail', 'Heavy Drizzle', 'Showers in the Vicinity', 'Light Rain Shower', 'Light Rain with Thunder', 'Mostly Cloudy / Windy', 'Partly Cloudy / Windy', 'Thunder in the Vicinity', 'T-Storm', 'Heavy Thunderstorms and Rain', 'Thunderstorm', 'Thunder', 'Heavy T-Storm', 'Funnel Cloud', 'Heavy T-Storm / Windy', 'Blowing Snow', 'Light Thunderstorms and Snow', 'Heavy Snow', 'Low Drifting Snow', 'Light

In [24]:
# 通用天气来代替
df.loc[df["Weather_Condition"].str.contains("Thunder|T-Storm", na=False), "Weather_Condition"] = "Thunderstorm"
df.loc[df["Weather_Condition"].str.contains("Snow|Sleet|Wintry", na=False), "Weather_Condition"] = "Snow"
df.loc[df["Weather_Condition"].str.contains("Rain|Drizzle|Shower", na=False), "Weather_Condition"] = "Rain"
df.loc[df["Weather_Condition"].str.contains("Wind|Squalls", na=False), "Weather_Condition"] = "Windy"
df.loc[df["Weather_Condition"].str.contains("Hail|Pellets", na=False), "Weather_Condition"] = "Hail"
df.loc[df["Weather_Condition"].str.contains("Fair", na=False), "Weather_Condition"] = "Clear"
df.loc[df["Weather_Condition"].str.contains("Cloud|Overcast", na=False), "Weather_Condition"] = "Cloudy"
df.loc[df["Weather_Condition"].str.contains("Mist|Haze|Fog", na=False), "Weather_Condition"] = "Fog"
df.loc[df["Weather_Condition"].str.contains("Sand|Dust", na=False), "Weather_Condition"] = "Sand"
df.loc[df["Weather_Condition"].str.contains("Smoke|Volcanic Ash", na=False), "Weather_Condition"] = "Smoke"
df.loc[df["Weather_Condition"].str.contains("N/A Precipitation", na=False), "Weather_Condition"] = np.nan

In [28]:
wc = pd.DataFrame(df['Weather_Condition'].value_counts()).reset_index().sort_values(by='Weather_Condition', ascending=False)
wc.rename(columns={'index':'weather_condition', 'Weather_Condition':'frequency'}, inplace=True)

In [30]:
wc.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/weather_all.csv", index=False)

### *交通*

In [31]:
road_features = ["Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit", "Railway", "Roundabout", "Station", "Stop", "Traffic_Calming", "Traffic_Signal"]

road_feat = df[road_features].sum().sort_values(ascending=False)

In [33]:
road_feat.to_csv("E:/desktop/2023win/visualization/DATA/Car/AfterCalculate/road_feature.csv", index=False)